## model set up

In [8]:
!pip install langchain --quiet

In [10]:
import warnings
import langchain
# langchain.__version__
import boto3
from langchain.chat_models import BedrockChat
from langchain.chains import NeptuneOpenCypherQAChain
from langchain.graphs import NeptuneGraph

In [12]:
host = "from cloudformation" # to be copied from the cloudformation e.g. cluter.cluster-xxxxxxxx.region.neptune.amazonaws.com
port = 8182
use_https = True

In [13]:
graph = NeptuneGraph(host=host, port=port, use_https=use_https)

In [17]:
client = boto3.client('bedrock-runtime', region_name='us-west-2')


llm = BedrockChat(
    model_id = "anthropic.claude-v2",
    client = client
)

chain = NeptuneOpenCypherQAChain.from_llm(
    llm = llm, graph = graph, verbose = True, top_K = 10, return_intermediate_steps=True, return_direct=False
)

# Explore edges

In [18]:
chain.run("how many edges between the users and the cells in the graph?")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (u:user)-[r:user_live_cell]->(c:cell)
RETURN COUNT(r) AS num_edges

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'num_edges': 1754981}]}

> Finished chain.


'Based on the provided information, there are 1754981 edges between the users and the cells in the graph.'

In [20]:
chain.run("how many vertexes of type user")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (u:user)
RETURN count(u) as num_users

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'num_users': 5002}]}

> Finished chain.


'Based on the provided information, the results show that there are 5002 users.'

In [21]:
chain.run("how many nodes of type cell")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (n:cell) RETURN count(n)

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'count(n)': 69000}]}

> Finished chain.


'Based on the information provided, the results indicate that there are 69000 nodes of type cell.'

In [22]:
chain.run("how many vertexes of type gnodeb")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (n:gnodeb) RETURN count(n)

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'count(n)': 5000}]}

> Finished chain.


'Based on the provided information, the results show that there are 5000 vertexes of type gnodeb.'

In [26]:
chain.run("list the two types of edges")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH ()-[r]->() 
RETURN DISTINCT TYPE(r) AS edge_type
LIMIT 2

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'edge_type': 'user_live_cell'}, {'edge_type': 'cell_relatedto_gnodeb'}]}

> Finished chain.


'Based on the provided information, the two types of edges are:\n\nuser_live_cell\ncell_relatedto_gnodeb'

In [27]:
chain.run("count the edges between the cell and gnodeb ?")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (u:user)-[r:user_live_cell]->(c:cell)-[rc:cell_relatedto_gnodeb]->(g:gnodeb)
RETURN COUNT(rc) AS edge_count

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'edge_count': 1754981}]}

> Finished chain.


'Based on the provided information, the edge count between the cell and gnodeb is 1754981.'

In [28]:
chain.run("count how many edges for cell_relatedto_gnodeb and user_live_cell")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (u:user)-[r1:user_live_cell]->(c:cell), (c)-[r2:cell_relatedto_gnodeb]->(g:gnodeb)
RETURN count(r1) AS user_live_cell_edges, count(r2) AS cell_relatedto_gnodeb_edges

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'user_live_cell_edges': 1754981, 'cell_relatedto_gnodeb_edges': 1754981}]}

> Finished chain.


'Based on the provided information, there are 1754981 edges for both cell_relatedto_gnodeb and user_live_cell.'

In [32]:
chain.run("what are the types of vertex ?")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (n)
RETURN DISTINCT labels(n) AS vertex_types

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': [{'vertex_types': ['user']}, {'vertex_types': ['cell']}, {'vertex_types': ['gnodeb']}]}

> Finished chain.


'Based on the provided information, the types of vertex are:\n\n- user\n- cell \n- gnodeb'

In [34]:
chain.run("to which cells the user_1500 is connceted")



> Entering new NeptuneOpenCypherQAChain chain...
Generated Cypher:
MATCH (u:user {name: "user_1500"})-[:user_live_cell]->(c:cell)
RETURN c

Full Context:
{'ResponseMetadata': {'HTTPStatusCode': 200, 'HTTPHeaders': {'transfer-encoding': 'chunked', 'content-type': 'application/json;charset=UTF-8'}, 'RetryAttempts': 0}, 'results': []}

> Finished chain.


"Based on the provided information, I don't know the answer. The provided information is empty."